In [2]:
import requests
import pandas as pd
import time

def get_market_codes():
    url = "https://api.upbit.com/v1/market/all"
    response = requests.get(url)
    markets = response.json()

    # Extract market codes
    market_codes = [market['market'] for market in markets if market['market'].startswith('KRW')]

    return market_codes

### Williams %R 계산

In [15]:
def calculate_williams_r(data):
    # Create DataFrame from the data
    df = pd.DataFrame(data)

    # Check if DataFrame is empty
    if df.empty:
        return None

    # Reverse the order (to have oldest to newest)
    df = df.iloc[::-1]

    # Calculate Williams %R
    high = df['high_price'].rolling(window=14).max()
    low = df['low_price'].rolling(window=14).min()
    williams_r = ((high - df['trade_price']) / (high - low)) * -100

    return williams_r.iloc[-1]

In [12]:
def get_over_sold_markets_on_hours(market_codes, threshold=-80):
    oversold_markets = []
    for code in market_codes:
        try:
            # Fetch market data
            url = f"https://api.upbit.com/v1/candles/minutes/60?market={code}&count=14"
            response = requests.get(url)
            data = response.json()

            # Calculate Williams %R for the market
            williams_r = calculate_williams_r(data)
            if williams_r is not None and williams_r <= threshold:
                oversold_markets.append((code, williams_r))

            time.sleep(0.1)
        except Exception as e:
            # Handle potential errors in data retrieval or calculation
            print(f"Error processing {code}: {e}")

    return oversold_markets

def get_over_sold_markets_on_days(market_codes, threshold=-80):
    oversold_markets = []
    for code in market_codes:
        try:
            # Fetch market data
            url = f"https://api.upbit.com/v1/candles/days?market={code}&count=14"
            response = requests.get(url)
            data = response.json()

            # Calculate Williams %R for the market
            williams_r = calculate_williams_r(data)
            if williams_r is not None and williams_r <= threshold:
                oversold_markets.append((code, williams_r))

            time.sleep(0.1)
        except Exception as e:
            # Handle potential errors in data retrieval or calculation
            print(f"Error processing {code}: {e}")

    return oversold_markets

In [16]:
def get_transitioning_markets(market_codes, lower_threshold=-80, upper_threshold=-80):
    transitioning_markets = []
    for code in market_codes:
        previous_williams_r = None  # Initialize a variable to store the previous day's Williams %R

        try:
            # Fetch market data for the last two days
            url = f"https://api.upbit.com/v1/candles/days?market={code}&count=15"
            response = requests.get(url)
            data = response.json()

            for daily_data in data:
                # Calculate Williams %R for each day
                williams_r = calculate_williams_r([daily_data])  # Calculate for a single day

                if previous_williams_r is not None:
                    # Check the transition condition
                    if previous_williams_r <= lower_threshold and williams_r > upper_threshold:
                        transitioning_markets.append((code, williams_r))
                        break  # Stop checking after finding the transition

                previous_williams_r = williams_r  # Update the previous value for the next iteration

            time.sleep(1)
        except Exception as e:
            print(f"Error processing {code}: {e}")

    return transitioning_markets

In [17]:
# 전체 마켓 코드를 가져온다. 
market_codes = get_market_codes()

codes_to_buy = get_transitioning_markets(market_codes)

# 1시간 기준으로 과매도 종목 구함.
# oversold_markets_on_hours = get_over_sold_markets_on_hours(market_codes)
# oversold_markets_on_days = get_over_sold_markets_on_days(market_codes)
oversold_markets_on_days

[('KRW-STORJ', -94.5910290237467),
 ('KRW-GAS', -86.30705394190872),
 ('KRW-ELF', -97.61904761904762),
 ('KRW-BTT', -80.0),
 ('KRW-DOGE', -84.61538461538461),
 ('KRW-AVAX', -81.09393579072533)]

In [18]:
codes_to_buy

[]

In [51]:
### 매수 : 이전 williams -80 미만 -> -80이상으로 변경된 종목
### 매도 : 현재 보유중인 종목 중 Williams %R 기준 -20 초과였다가 -20 이하로 변경된 종목

In [82]:
### 내 보유 종목 가져오기 
import os
import uuid
import jwt
from dotenv import load_dotenv

load_dotenv()

def get_jwt_token():
    access_key = os.getenv('UPBIT_OPEN_API_ACCESS_KEY')
    secret_key = os.getenv('UPBIT_OPEN_API_SECRET_KEY')

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
    }

    return jwt.encode(payload, secret_key, algorithm='HS256')

# 내 종목 구하기
def get_my_holdings():
    jwt_token = get_jwt_token()
    headers = {'Authorization': f'Bearer {jwt_token}'}
    res = requests.get('https://api.upbit.com/v1/accounts', headers=headers)
    return res.json()

# 종목 매수 
def buy_market_order(market, price):
    jwt_token = get_jwt_token()
    headers = {'Authorization': f'Bearer {jwt_token}'}

    data = {
        'market': market,
        'side': 'bid',
        'price': str(price),
        'ord_type': 'price',
    }

    response = requests.post('https://api.upbit.com/v1/orders', headers=headers, data=data)
    return response.json()


# 종목 매도 
def sell_market_order(market, volume):
    jwt_token = get_jwt_token()
    print(jwt_token)
    headers = {'Authorization': f'Bearer {jwt_token}'}

    data = {
        'market': market,
        'side': 'ask',
        'volume': str(volume),
        'ord_type': 'market',
    }

    response = requests.post('https://api.upbit.com/v1/orders', headers=headers, data=data)
    return response.json()